In [1]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 965.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/NIR/data(markup 1, 2, 3, 5).csv" /content/train.csv

In [4]:
!cp "/content/drive/MyDrive/NIR/bald-20230413T104914Z-001.zip" "/content/bald-20230413T104914Z-001.zip"
!cp "/content/drive/MyDrive/NIR/bald-20230413T104914Z-002.zip" "/content/bald-20230413T104914Z-002.zip"

In [5]:
!unzip "/content/bald-20230413T104914Z-001.zip" 
!unzip "/content/bald-20230413T104914Z-002.zip"

Archive:  /content/bald-20230413T104914Z-001.zip
  inflating: bald/optimizer.pt       
  inflating: bald/rng_state.pth      
  inflating: bald/config.json        
  inflating: bald/scheduler.pt       
  inflating: bald/trainer_state.json  
  inflating: bald/training_args.bin  
Archive:  /content/bald-20230413T104914Z-002.zip
  inflating: bald/pytorch_model.bin  


In [6]:
import pandas as pd

train = pd.read_csv("/content/train.csv", encoding='utf-8')

In [7]:
import os
import random
from copy import deepcopy

import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [8]:
data = load_dataset("csv", data_files={'train': '/content/train.csv'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-630cfd25bdd31917/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained("/content/bald")

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=2,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = BaalTransformersTrainer(
    model=model,
    args=training_args,
    tokenizer=None
)

In [10]:
def tokenize_function(examples):
    return tokenizer(examples['first_part_content'], examples['part_body'], max_length=512, truncation=True, padding="max_length")

In [11]:
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/92995 [00:00<?, ? examples/s]

In [12]:
output = trainer.predict(tokenized_data['train'])

In [13]:
predict_labels = np.argmax(output.predictions, axis=1)

In [14]:
train['relevance'] = predict_labels
train

,match_id,id,team1,team2,broadcast_name,first_part_content,last_part_content,part_body,after_match,relevance
0,18925,7267034,Локомотив,Химки,Россия. Премьер-лига,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...,Три тысячи сотрудников милиции будут следить з...,0,0
1,18925,7269714,Локомотив,Химки,Россия. Премьер-лига,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...,Текстовый онлайн матча «Атлетик» – «Реал» – на...,0,0
2,18925,7270558,Локомотив,Химки,Россия. Премьер-лига,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...,Премьер-лига. 1-й тур. «Терек» обыграл «Спарта...,0,0
3,18925,7274151,Локомотив,Химки,Россия. Премьер-лига,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...,Дюрица получил травму в игре с «Химками». Защи...,1,1
4,18925,7274661,Локомотив,Химки,Россия. Премьер-лига,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...,Константин Сарсания: «Если бы выпустил второго...,1,0
...,...,...,...,...,...,...,...,...,...,...
92990,1104866,1040642045,Питтсбург,Сан-Хосе,Кубок Стэнли,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...",Майк Салливан: «Атака – козырь «Сан-Хосе» и мы...,1,0
92991,1104866,1040648050,Питтсбург,Сан-Хосе,Кубок Стэнли,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...","Конор Шери: «Кросби сказал, что выиграет вбрас...",1,0
92992,1104866,1040648436,Питтсбург,Сан-Хосе,Кубок Стэнли,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...",Сидни Кросби: «Победным голом мы больше обязан...,1,0
92993,1104866,1040653602,Питтсбург,Сан-Хосе,Кубок Стэнли,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...",Джастин Браун: «Питтсбург» все наступал и наст...,1,0


In [15]:
train['relevance'].value_counts()

0    77905
1    15090
Name: relevance, dtype: int64

In [16]:
train.to_csv('data.csv', index=False, encoding='utf-8')

In [17]:
!zip -r "/content/predicted_data(markup 1, 2, 3, 5).zip" "/content/data.csv"
!cp "/content/predicted_data(markup 1, 2, 3, 5).zip" "/content/drive/MyDrive/NIR/predicted_data(markup 1, 2, 3, 5).zip"

  adding: content/data.csv (deflated 92%)
